In [1]:
# base
import re
import numpy as np

# tensorflow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:


with open('data/biden_raw_text.txt', 'r', encoding='utf8') as myfile:
    raw_text = myfile.read()
    
def clean_text(t):
    # to lower
    t = t.lower()
    # remove quotes
    t = re.sub(r'"@.*', '', t)
    t = re.sub(r'^“.*”$', '', t)
    # remove URLs
    t = re.sub(r'https*:\/\/\S*', '', t)
    t = re.sub(r'pic\.twitter\.com\/\S*', '', t)
    # remove \n
    t = re.sub('\n', ' ', t)
    # remove extra whitespaces
    t = re.sub(r'\s+', ' ', t)
    # replace '&amp' with 'and'
    t = re.sub('&amp;', 'and', t)     
    # replace abbreviations
    t = re.sub("'ll", ' will', t)
    t = re.sub("won't", 'will not', t)
    t = re.sub("n't", ' not', t) 
    # remove @mention
    t = re.sub(r'@[A-Za-z0-9_]+', '', t) 
    # remove #tag
    t = re.sub(r'#[A-Za-z0-9_]+', '', t) 
    # remove special characters
    t = re.sub(r'[^a-zA-Z ]', '', t) 
    # remove multiple spaces 
    t = re.sub("\s\s+", " ", t) 
    return t

raw_text = clean_text(raw_text)

In [3]:
raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  574504
Total Vocab:  27


In [4]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

In [5]:
# reshape
X = np.reshape(dataX, (len(dataX), seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)

In [6]:
from tensorflow.keras.models import load_model

model = load_model('models/trump_model.h5')

In [7]:
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model_save = ModelCheckpoint('models/biden_model.h5', save_best_only=True, monitor='val_loss', mode='min')
# fit the model
model.fit(X, y, epochs=100, batch_size=128, callbacks=[early_stop, model_save], validation_split=0.1)

Train on 516963 samples, validate on 57441 samples
Epoch 1/100
516963/516963 [==============================] - 610s 1ms/sample - loss: 1.3796 - val_loss: 1.2614
Epoch 2/100
516963/516963 [==============================] - 623s 1ms/sample - loss: 1.3022 - val_loss: 1.2263
Epoch 3/100
516963/516963 [==============================] - 647s 1ms/sample - loss: 1.2677 - val_loss: 1.2039
Epoch 4/100
516963/516963 [==============================] - 641s 1ms/sample - loss: 1.2430 - val_loss: 1.1972
Epoch 5/100
516963/516963 [==============================] - 641s 1ms/sample - loss: 1.2235 - val_loss: 1.1893
Epoch 6/100
516963/516963 [==============================] - 662s 1ms/sample - loss: 1.2087 - val_loss: 1.1829
Epoch 7/100
516963/516963 [==============================] - 667s 1ms/sample - loss: 1.1953 - val_loss: 1.1700
Epoch 8/100
516963/516963 [==============================] - 647s 1ms/sample - loss: 1.1844 - val_loss: 1.1649
Epoch 9/100
516963/516963 [==============================] - 

KeyboardInterrupt: 